In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# 线性回归的import
from sklearn.linear_model import LinearRegression

# 逻辑回归的import
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# 神经网络的import
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam

In [2]:
url = 'https://drive.google.com/file/d/1EHmptHyzY8owv0BAcNKtkQpMwfkURwRy/view?usp=share_link'
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
dataset = pd.read_csv(path)

C:\Users\11439\AppData\Local\Temp\ipykernel_23280\3463052686.py:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(path)


In [3]:
dataset

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,...,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
0,ESPORTSTMNT01_2690210,complete,NaN,LCKC,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,121.0,391.0,345.0,14.0,0.0,1.0,0.0,0.0,1.0,0.0
1,ESPORTSTMNT01_2690210,complete,NaN,LCKC,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,100.0,541.0,-275.0,-11.0,2.0,3.0,2.0,0.0,5.0,1.0
2,ESPORTSTMNT01_2690210,complete,NaN,LCKC,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,119.0,-475.0,153.0,1.0,0.0,3.0,0.0,3.0,3.0,2.0
3,ESPORTSTMNT01_2690210,complete,NaN,LCKC,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,149.0,-793.0,-1343.0,-34.0,2.0,1.0,2.0,3.0,3.0,0.0
4,ESPORTSTMNT01_2690210,complete,NaN,LCKC,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,21.0,443.0,-497.0,7.0,1.0,2.0,2.0,0.0,6.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149395,9687-9687_game_5,partial,https://lpl.qq.com/es/stats.shtml?bmid=9687,DCup,2022,NaN,0,2022-12-27 12:43:43,5,12.23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149396,9687-9687_game_5,partial,https://lpl.qq.com/es/stats.shtml?bmid=9687,DCup,2022,NaN,0,2022-12-27 12:43:43,5,12.23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149397,9687-9687_game_5,partial,https://lpl.qq.com/es/stats.shtml?bmid=9687,DCup,2022,NaN,0,2022-12-27 12:43:43,5,12.23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149398,9687-9687_game_5,partial,https://lpl.qq.com/es/stats.shtml?bmid=9687,DCup,2022,NaN,0,2022-12-27 12:43:43,5,12.23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 可能和胜利挂钩的一些参数：

A:	Total assists\
D:	Total deaths

DRG%:	Dragon control rate: percent of all Dragons killed that were taken by the team, reflecting only elemental drakes if ELD% is present (控龙率)\
ELD%:	Elder dragon control rate(远古龙控制率)\
FBN%：	First Baron rate(首条大龙率)\
FD:%	First dragon rat(首龙概率)\
HLD%:	Rift Herald control rate(峡谷先锋控制率)
e
F:T%	First tower ra(一塔概率)\
F3T%	First-to-three-towers rate (percentage of games in which team was the first to 3 tower kills(摧毁三个塔的概率)\
FBV%:	First Blood Victim rate -- percent of games player/champion was killed for First Blood(一血概率)

GD20:	Average gold difference at 20 minutes(20分钟平均经济差)\
GPM：	Average gold per minute(分均经济)\
VSPM:	Vision score per minute(分均视野得分)

KDA:	Total Kill/Death/Assist ratio

W：	Wins

te

In [4]:
# 所有column名称
# for col in dataset.columns:
#     print(col)

## Data cleanning

In [5]:
variables = ["league", "teamname", "position", "side",
             "firstblood", "firsttower", "firstdragon", "firsttothreetowers",
             "killsat15", "deathsat15", "golddiffat15","result"]

In [6]:
dataset_var = dataset[variables]
dataset_var

,league,teamname,position,side,firstblood,firsttower,firstdragon,firsttothreetowers,killsat15,deathsat15,golddiffat15,result
0,LCKC,Fredit BRION Challengers,top,Blue,0.0,NaN,NaN,NaN,0.0,0.0,391.0,0
1,LCKC,Fredit BRION Challengers,jng,Blue,1.0,NaN,NaN,NaN,2.0,2.0,541.0,0
2,LCKC,Fredit BRION Challengers,mid,Blue,0.0,NaN,NaN,NaN,0.0,0.0,-475.0,0
3,LCKC,Fredit BRION Challengers,bot,Blue,1.0,NaN,NaN,NaN,2.0,2.0,-793.0,0
4,LCKC,Fredit BRION Challengers,sup,Blue,1.0,NaN,NaN,NaN,1.0,2.0,443.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
149395,DCup,Bilibili Gaming,mid,Red,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
149396,DCup,Bilibili Gaming,bot,Red,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
149397,DCup,Bilibili Gaming,sup,Red,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
149398,DCup,ThunderTalk Gaming,team,Blue,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0


预测胜利与否，通过辅助的视野得分，团队的15分钟的经济差（最终的经济参考价值不大），是否有一血，是否一塔，是否首龙，红蓝方

In [7]:
team_data = dataset_var[dataset_var['position'] == 'team']
team_data

,league,teamname,position,side,firstblood,firsttower,firstdragon,firsttothreetowers,killsat15,deathsat15,golddiffat15,result
10,LCKC,Fredit BRION Challengers,team,Blue,1.0,1.0,0.0,1.0,5.0,6.0,107.0,0
11,LCKC,Nongshim RedForce Challengers,team,Red,0.0,0.0,1.0,0.0,6.0,5.0,-107.0,1
22,LCKC,T1 Challengers,team,Blue,0.0,0.0,0.0,0.0,1.0,3.0,-1763.0,0
23,LCKC,Liiv SANDBOX Challengers,team,Red,1.0,1.0,1.0,1.0,3.0,1.0,1763.0,1
34,LPL,Oh My God,team,Blue,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...
149375,DCup,Bilibili Gaming,team,Red,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0
149386,DCup,Bilibili Gaming,team,Blue,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1
149387,DCup,ThunderTalk Gaming,team,Red,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0
149398,DCup,ThunderTalk Gaming,team,Blue,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0


In [8]:
nan_counts = team_data.isna().sum()
nan_counts

league                   0
teamname                 9
position                 0
side                     0
firstblood               2
firsttower            3638
firstdragon           3638
firsttothreetowers    3638
killsat15             3638
deathsat15            3638
golddiffat15          3638
result                   0
dtype: int64

In [9]:
cleaned_team_data = team_data.dropna().reset_index(drop = True)
cleaned_team_data.head(6)

,league,teamname,position,side,firstblood,firsttower,firstdragon,firsttothreetowers,killsat15,deathsat15,golddiffat15,result
0,LCKC,Fredit BRION Challengers,team,Blue,1.0,1.0,0.0,1.0,5.0,6.0,107.0,0
1,LCKC,Nongshim RedForce Challengers,team,Red,0.0,0.0,1.0,0.0,6.0,5.0,-107.0,1
2,LCKC,T1 Challengers,team,Blue,0.0,0.0,0.0,0.0,1.0,3.0,-1763.0,0
3,LCKC,Liiv SANDBOX Challengers,team,Red,1.0,1.0,1.0,1.0,3.0,1.0,1763.0,1
4,LCKC,KT Rolster Challengers,team,Blue,0.0,1.0,1.0,1.0,3.0,1.0,1191.0,1
5,LCKC,Gen.G Challengers,team,Red,1.0,0.0,0.0,0.0,1.0,3.0,-1191.0,0


## LinearRegression

In [10]:
features = ['side', 'firstblood', 'firsttower', 'firstdragon', 'firsttothreetowers', 
            'killsat15', 'golddiffat15']
X = cleaned_team_data[features]
y = cleaned_team_data['result']

# 将side进行one-hot encoding
categorical_features = ['side']
one_hot = OneHotEncoder()
transformer = ColumnTransformer([("one_hot", one_hot, categorical_features)],
                                remainder='passthrough')
X = transformer.fit_transform(X)

# 分割数据集为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [11]:
# 训练一个线性回归模型
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

# 使用模型进行预测
y_pred_continuous = lin_reg.predict(X_test)

# 将连续输出转换为二元类别
y_pred_class = (y_pred_continuous >= 0.5).astype(int)

# 计算准确率
accuracy = accuracy_score(y_test, y_pred_class)
print("Accuracy: ", accuracy)

Accuracy:  0.7804265997490589


## LogisticRegression

In [12]:
# 选择特征和目标变量为游戏中期数据
features = ['side', 'firstblood', 'firsttower', 'firstdragon', 'firsttothreetowers', 
            'killsat15', 'golddiffat15'] #vision score值得注意
X = cleaned_team_data[features]
y = cleaned_team_data['result']

# 将side进行one-hot encoding
categorical_features = ['side']
one_hot = OneHotEncoder()
transformer = ColumnTransformer([("one_hot", one_hot, categorical_features)],
                                remainder='passthrough')

In [13]:
log_reg = LogisticRegression(max_iter=1000)

pipeline = Pipeline(steps=[('transformer', transformer), ('model', log_reg)])


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
pipeline.fit(X_train, y_train)

# 预测测试集
y_pred = pipeline.predict(X_test)

# 计算准确率
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)

Accuracy:  0.7787013801756587


## Neural Networks

In [14]:
features = ['side', 'firstblood', 'firsttower', 'firstdragon', 'firsttothreetowers', 
            'killsat15', 'golddiffat15']
X = cleaned_team_data[features]
y = cleaned_team_data['result']

one_hot = OneHotEncoder()
X_categorical = one_hot.fit_transform(X[['side']]).toarray()

# 将one-hot结果和其他数据合并
X_numerical = X.drop('side', axis=1)
X = np.hstack((X_categorical, X_numerical))

# 分割数据集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [15]:
# 创建神经网络模型
model = Sequential()
model.add(Dense(16, input_dim=X_train.shape[1]))
model.add(Activation('relu'))
model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [16]:
# 编译
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# 训练
model.fit(X_train, y_train, epochs=30, batch_size=32)

Epoch 1/30


465/465 [==============================] - 4s 3ms/step - loss: 9.8597 - accuracy: 0.6786
Epoch 2/30
465/465 [==============================] - 1s 3ms/step - loss: 1.2893 - accuracy: 0.6720
Epoch 3/30
465/465 [==============================] - 1s 3ms/step - loss: 0.9645 - accuracy: 0.6879
Epoch 4/30
465/465 [==============================] - 2s 3ms/step - loss: 1.1510 - accuracy: 0.6861
Epoch 5/30
465/465 [==============================] - 1s 3ms/step - loss: 1.1051 - accuracy: 0.6932
Epoch 6/30
465/465 [==============================] - 1s 2ms/step - loss: 0.9264 - accuracy: 0.6915
Epoch 7/30
465/465 [==============================] - 1s 3ms/step - loss: 0.8603 - accuracy: 0.6978
Epoch 8/30
465/465 [==============================] - 1s 3ms/step - loss: 0.9818 - accuracy: 0.6995
Epoch 9/30
465/465 [==============================] - 1s 3ms/step - loss: 0.9462 - accuracy: 0.6982
Epoch 10/30
465/465 [==============================] - 1s 3ms/step - loss: 0.9238 - accuracy: 0.71

In [17]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Accuracy: ", accuracy)

200/200 [==============================] - 1s 2ms/step - loss: 0.5213 - accuracy: 0.7749
Accuracy:  0.7749372720718384


In [18]:
cleaned_team_data.head(8)

,league,teamname,position,side,firstblood,firsttower,firstdragon,firsttothreetowers,killsat15,deathsat15,golddiffat15,result
0,LCKC,Fredit BRION Challengers,team,Blue,1.0,1.0,0.0,1.0,5.0,6.0,107.0,0
1,LCKC,Nongshim RedForce Challengers,team,Red,0.0,0.0,1.0,0.0,6.0,5.0,-107.0,1
2,LCKC,T1 Challengers,team,Blue,0.0,0.0,0.0,0.0,1.0,3.0,-1763.0,0
3,LCKC,Liiv SANDBOX Challengers,team,Red,1.0,1.0,1.0,1.0,3.0,1.0,1763.0,1
4,LCKC,KT Rolster Challengers,team,Blue,0.0,1.0,1.0,1.0,3.0,1.0,1191.0,1
5,LCKC,Gen.G Challengers,team,Red,1.0,0.0,0.0,0.0,1.0,3.0,-1191.0,0
6,LCKC,DWG KIA Challengers,team,Blue,1.0,1.0,0.0,1.0,4.0,2.0,550.0,0
7,LCKC,DRX Challengers,team,Red,0.0,0.0,1.0,0.0,2.0,4.0,-550.0,1


In [19]:
#需要采用 killsat15 - deathsat15 的差值来表示diff

In [20]:
# 计算 15 分钟时的击杀和死亡数差值
cleaned_team_data['kills_deaths_diff_at15'] = cleaned_team_data['killsat15'] - cleaned_team_data['deathsat15']
cleaned_team_data.drop(['killsat15', 'deathsat15'], axis=1, inplace=True)
# 获取所有列的名称
columns = cleaned_team_data.columns.tolist()

# 移除 'result' 和 'kills_deaths_diff_at15'
columns.remove('result')
columns.remove('kills_deaths_diff_at15')

# 重新排列列，使 result在最右侧
cleaned_team_data = cleaned_team_data[columns + ['kills_deaths_diff_at15', 'result']]
cleaned_team_data

,league,teamname,position,side,firstblood,firsttower,firstdragon,firsttothreetowers,golddiffat15,kills_deaths_diff_at15,result
0,LCKC,Fredit BRION Challengers,team,Blue,1.0,1.0,0.0,1.0,107.0,-1.0,0
1,LCKC,Nongshim RedForce Challengers,team,Red,0.0,0.0,1.0,0.0,-107.0,1.0,1
2,LCKC,T1 Challengers,team,Blue,0.0,0.0,0.0,0.0,-1763.0,-2.0,0
3,LCKC,Liiv SANDBOX Challengers,team,Red,1.0,1.0,1.0,1.0,1763.0,2.0,1
4,LCKC,KT Rolster Challengers,team,Blue,0.0,1.0,1.0,1.0,1191.0,2.0,1
...,...,...,...,...,...,...,...,...,...,...,...
21248,NEXO,unknown team,team,Red,0.0,1.0,1.0,0.0,-853.0,2.0,1
21249,NEXO,unknown team,team,Blue,1.0,1.0,0.0,1.0,2063.0,3.0,1
21250,NEXO,Córdoba Patrimonio eSports,team,Red,0.0,0.0,1.0,0.0,-2063.0,-3.0,0
21251,NEXO,unknown team,team,Blue,1.0,1.0,1.0,1.0,2213.0,1.0,1


In [21]:
cleaned_team_data.head(6)

,league,teamname,position,side,firstblood,firsttower,firstdragon,firsttothreetowers,golddiffat15,kills_deaths_diff_at15,result
0,LCKC,Fredit BRION Challengers,team,Blue,1.0,1.0,0.0,1.0,107.0,-1.0,0
1,LCKC,Nongshim RedForce Challengers,team,Red,0.0,0.0,1.0,0.0,-107.0,1.0,1
2,LCKC,T1 Challengers,team,Blue,0.0,0.0,0.0,0.0,-1763.0,-2.0,0
3,LCKC,Liiv SANDBOX Challengers,team,Red,1.0,1.0,1.0,1.0,1763.0,2.0,1
4,LCKC,KT Rolster Challengers,team,Blue,0.0,1.0,1.0,1.0,1191.0,2.0,1
5,LCKC,Gen.G Challengers,team,Red,1.0,0.0,0.0,0.0,-1191.0,-2.0,0


In [22]:
# 红色方和蓝色方的胜率差异

In [23]:
# 计算红蓝方胜率
win_rates = cleaned_team_data.groupby('side')['result'].mean()
print(win_rates)

side
Blue    0.522779
Red     0.477091
Name: result, dtype: float64


In [24]:
# 用permutation test来验证这个胜率差异是否具有统计学的意义

In [25]:
# 提取蓝色方和红色方的胜利数据
blue_wins = cleaned_team_data[cleaned_team_data['side'] == 'Blue']['result']
red_wins = cleaned_team_data[cleaned_team_data['side'] == 'Red']['result']

# 计算原始差异
original_diff = np.abs(blue_wins.mean() - red_wins.mean())
original_diff

0.04568763694348382

In [26]:
# 置换检验
n_permutations = 10000
count = 0

for _ in range(n_permutations):
    # 合并数据并随机置换
    combined = np.random.permutation(np.concatenate((blue_wins, red_wins)))

    # 分成新的两组
    new_blue_wins = combined[:len(blue_wins)]
    new_red_wins = combined[len(blue_wins):]

    # 计算新的差异
    new_diff = np.abs(new_blue_wins.mean() - new_red_wins.mean())

    # 检查新差异是否大于或等于原始差异
    if new_diff >= original_diff:
        count += 1

# 计算 p 值
p_value = count / n_permutations
print(f"p-value: {p_value}")

p-value: 0.0


In [27]:
# p = 0 说明side是具有统计学意义的，它是会影响result的

In [28]:
#######################################################################################

In [29]:
# 线性回归：

In [30]:
features = ['side', 'firstblood', 'firsttower', 'firstdragon', 'firsttothreetowers', 'golddiffat15', 'kills_deaths_diff_at15']
X = cleaned_team_data[features]
y = cleaned_team_data['result']

# 对分类特征 'side' 进行独热编码
X = pd.get_dummies(X, columns=['side'], drop_first=True)

# 分割数据集为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 训练一个线性回归模型
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

# 使用模型进行预测
y_pred_continuous = lin_reg.predict(X_test)

# 将连续输出转换为二元类别
y_pred_class = (y_pred_continuous >= 0.5).astype(int)

# 计算准确率
accuracy = accuracy_score(y_test, y_pred_class)
print("Accuracy: ", accuracy)

Accuracy:  0.779642409033877


In [31]:
# 逻辑回归

In [32]:
features = ['side', 'firstblood', 'firsttower', 'firstdragon', 'firsttothreetowers', 'golddiffat15', 'kills_deaths_diff_at15']
X = cleaned_team_data[features]
y = cleaned_team_data['result']

# 对分类特征 'side' 进行独热编码
X = pd.get_dummies(X, columns=['side'], drop_first=True)

# 分割数据集为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 创建逻辑回归模型
log_reg = LogisticRegression(max_iter=1000)

# 训练模型
log_reg.fit(X_train, y_train)

# 预测测试集
y_pred = log_reg.predict(X_test)

# 计算准确率
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)

Accuracy:  0.7785445420326224


In [33]:
features = ['side', 'firstblood', 'firsttower', 'firstdragon', 'firsttothreetowers', 'golddiffat15', 'kills_deaths_diff_at15']
X = cleaned_team_data[features]
y = cleaned_team_data['result']

one_hot = OneHotEncoder()
X_categorical = one_hot.fit_transform(X[['side']]).toarray()

# 将one-hot结果和其他数据合并
X_numerical = X.drop('side', axis=1)
X = np.hstack((X_categorical, X_numerical))

# 分割数据集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [34]:
# 创建神经网络模型
model = Sequential()
model.add(Dense(16, input_dim=X_train.shape[1]))
model.add(Activation('relu'))
model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [35]:
# 编译
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# 训练
model.fit(X_train, y_train, epochs=30, batch_size=32)

Epoch 1/30
465/465 [==============================] - 3s 4ms/step - loss: 10.6021 - accuracy: 0.6684
Epoch 2/30
465/465 [==============================] - 1s 2ms/step - loss: 1.3568 - accuracy: 0.6770
Epoch 3/30
465/465 [==============================] - 1s 3ms/step - loss: 1.3009 - accuracy: 0.6735
Epoch 4/30
465/465 [==============================] - 1s 3ms/step - loss: 1.4870 - accuracy: 0.6770
Epoch 5/30
465/465 [==============================] - 1s 3ms/step - loss: 1.1343 - accuracy: 0.6895
Epoch 6/30
465/465 [==============================] - 1s 3ms/step - loss: 1.2120 - accuracy: 0.6765
Epoch 7/30
465/465 [==============================] - 1s 3ms/step - loss: 1.2475 - accuracy: 0.6848
Epoch 8/30
465/465 [==============================] - 1s 3ms/step - loss: 1.1343 - accuracy: 0.6842
Epoch 9/30
465/465 [==============================] - 1s 2ms/step - loss: 1.4333 - accuracy: 0.6817
Epoch 10/30
465/465 [==============================] - 1s 3ms/step - loss: 1.4059 - accuracy: 0.683

In [36]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Accuracy: ", accuracy)

200/200 [==============================] - 1s 3ms/step - loss: 0.5495 - accuracy: 0.7604
Accuracy:  0.760351300239563


In [37]:
team_data

,league,teamname,position,side,firstblood,firsttower,firstdragon,firsttothreetowers,killsat15,deathsat15,golddiffat15,result
10,LCKC,Fredit BRION Challengers,team,Blue,1.0,1.0,0.0,1.0,5.0,6.0,107.0,0
11,LCKC,Nongshim RedForce Challengers,team,Red,0.0,0.0,1.0,0.0,6.0,5.0,-107.0,1
22,LCKC,T1 Challengers,team,Blue,0.0,0.0,0.0,0.0,1.0,3.0,-1763.0,0
23,LCKC,Liiv SANDBOX Challengers,team,Red,1.0,1.0,1.0,1.0,3.0,1.0,1763.0,1
34,LPL,Oh My God,team,Blue,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...
149375,DCup,Bilibili Gaming,team,Red,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0
149386,DCup,Bilibili Gaming,team,Blue,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1
149387,DCup,ThunderTalk Gaming,team,Red,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0
149398,DCup,ThunderTalk Gaming,team,Blue,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0


In [38]:
def fill_binary_columns_randomly(df, columns):
    modified_df = df.copy()

    for i in range(0, len(modified_df), 2):
        row1_index, row2_index = i, i + 1
        for col in columns:
            if pd.isna(modified_df.iloc[row1_index][col]) and pd.isna(modified_df.iloc[row2_index][col]):
                value = np.random.choice([0, 1])
                modified_df.iloc[row1_index, modified_df.columns.get_loc(col)] = value
                modified_df.iloc[row2_index, modified_df.columns.get_loc(col)] = 1 - value
            elif pd.isna(modified_df.iloc[row1_index][col]):
                modified_df.iloc[row1_index, modified_df.columns.get_loc(col)] = 1 - modified_df.iloc[row2_index][col]
            elif pd.isna(modified_df.iloc[row2_index][col]):
                modified_df.iloc[row2_index, modified_df.columns.get_loc(col)] = 1 - modified_df.iloc[row1_index][col]

    return modified_df

columns_to_fill = ['firsttower', 'firstdragon', 'firsttothreetowers', 'killsat15', 'deathsat15', 'golddiffat15']

filled_team_data = fill_binary_columns_randomly(team_data, columns_to_fill)

In [39]:
filled_team_data.head(8)  # 显示前8行以验证结果

,league,teamname,position,side,firstblood,firsttower,firstdragon,firsttothreetowers,killsat15,deathsat15,golddiffat15,result
10,LCKC,Fredit BRION Challengers,team,Blue,1.0,1.0,0.0,1.0,5.0,6.0,107.0,0
11,LCKC,Nongshim RedForce Challengers,team,Red,0.0,0.0,1.0,0.0,6.0,5.0,-107.0,1
22,LCKC,T1 Challengers,team,Blue,0.0,0.0,0.0,0.0,1.0,3.0,-1763.0,0
23,LCKC,Liiv SANDBOX Challengers,team,Red,1.0,1.0,1.0,1.0,3.0,1.0,1763.0,1
34,LPL,Oh My God,team,Blue,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1
35,LPL,ThunderTalk Gaming,team,Red,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0
46,LCKC,KT Rolster Challengers,team,Blue,0.0,1.0,1.0,1.0,3.0,1.0,1191.0,1
47,LCKC,Gen.G Challengers,team,Red,1.0,0.0,0.0,0.0,1.0,3.0,-1191.0,0


In [40]:
filled_team_data.isna().sum()

league                0
teamname              9
position              0
side                  0
firstblood            2
firsttower            0
firstdragon           0
firsttothreetowers    0
killsat15             0
deathsat15            0
golddiffat15          0
result                0
dtype: int64

In [41]:
missing_firstblood_rows = filled_team_data[filled_team_data['firstblood'].isna()]

missing_firstblood_rows

,league,teamname,position,side,firstblood,firsttower,firstdragon,firsttothreetowers,killsat15,deathsat15,golddiffat15,result
17878,LDL,ThunderTalk Gaming Young,team,Blue,NaN,1.0,0.0,0.0,1.0,0.0,1.0,0
17879,LDL,TEAM ORANGE,team,Red,NaN,0.0,1.0,1.0,0.0,1.0,0.0,1


In [45]:
filled_team_data['golddiffat15'].mean()

0.07305220883534136

In [47]:
filled_team_data['deathsat15'].mean()

3.5604417670682733

In [48]:
filled_team_data['killsat15'].mean()

3.5518875502008034

In [49]:
filled_team_data['deathsat15'].mean() - filled_team_data['killsat15'].mean()

0.008554216867469933